# Pandas Introduction 
Material referenced from Data 100, EEP 153 Spring 2023 Materials.

Learning Goals:
1. DataFrame & Series
2. Slicing, iloc, loc
3. Groupby
4. Joins & Merges
5. Problem Solving

Intro Pandas DataFrames
Pandas: Slicing & Indexing
Pandas: Aggregation
Python Visualizations

Pandas is a data analysis library/toolkit for Python. It is widely used in data science because it allows us to create objects called DataFrames that are just like tables and can be manipulated as such. These are basically rectangular arrays of data, with names for rows and columns, rather like a spreadsheet. In fact, one important thing one can do with DataFrames is to import data from spreadsheets. They also integrate well with NumPy arrays!

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

## Intro to Pandas DataFrames

A [dataframe](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) is a table in which each column has a type; there is an index over the columns (typically string labels) and an index over the rows (typically ordinal numbers).

The [docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) for the pandas `DataFrame` class  provide at least two syntaxes to create a data frame.

Let's make our own DataFrame and learn to manipulate it. One way to make a DataFrame is to use pd.DataFrame and input the data manually, like below. Later, we'll show how to import Google Sheets or Excel files into DataFrames, which will be much more applicable in projects and assignments.

You can create a data frame by specifying the columns and values using a dictionary as shown below. 

The keys of the dictionary are the column names, and the values of the dictionary are lists containing the row entries.

In [ ]:
my_recipe = pd.DataFrame(
    data={'ingredient': ['eggs', 'brown sugar', 'flour', 'vanilla', 'sugar', 'chocolate chips', 'baking soda', 'butter'],
          'price': [3.5, 2.99, 5.49, 5, 3.99, 2.99, 4.50, 5.00]
          })
my_recipe


You can obtain the dimensions of a dataframe by using the shape attribute `dataframe.shape` in a tuple with (# of rows, # of columns).

In [ ]:
my_recipe.shape

You can also convert the entire dataframe into a two-dimensional numpy array using the `dataframe.values` for a DataFrame.

In [ ]:
my_recipe.values

You can add a column to a dataframe very easily with `d['new column name']` = ... and assign a list or array of values to the column.

In [ ]:
my_recipe['need to buy'] = [0,1,1,1,0,1,0,1]
my_recipe

You can also add a column to `d` with `d.loc[:, 'new column name'] = ...`. This way to modify an existing dataframe is faster and therefore preferred over the assignment syntax above. The first parameter is for the rows and second is for columns. The `:` means change all rows and the `new column name` indicates the column you are modifying (or in this case, adding). 


In [ ]:
my_recipe.loc[:,'required for recipe'] = ['y', 'y', 'y','y', 'y', 'n', 'y', 'y']
my_recipe

You can use the `.drop()` method to [drop](https://pandas.pydata.org/pandasdocs/stable/generated/pandas.DataFrame.drop.html) to drop any columns you no longer need in your dataframe.

In [ ]:
my_recipe = my_recipe.drop(columns = ['required for recipe'])
my_recipe

We can also use the `.rename()` method to [rename](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html) dataframe columns. Let's rename all our columns to be capitalized. 

In [ ]:
my_recipe = my_recipe.rename(columns = {'ingredient': 'Ingredient', 'price': 'Price', 'need to buy':'Need to Buy'})
my_recipe

Beisdes DataFrames, another useful Pandas structure is a [Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html). A Series is a type of list in Pandas which can take integer values, string values, and more that returns a Series object. It is different from a DataFrame, which is made up of multiple Series, while a single Series can only contain one list. Here's an example:

In [ ]:
ingredient_series = my_recipe['Ingredient']
ingredient_series

As you can see, a Series is an object that is a list of values indexed from 0 to n. We can think of DataFrames as enhanced numpy 2D arrays (with better visual representation), and Series as enhanced numpy 1D arrays.

In [ ]:
type(ingredient_series)

## Pandas: Slicing and Indexing

Now, we're going to look at some school census data from https://ww2.amstat.org/censusatschool/. Pandas has the capability of reading data from different file formats (such as .csv, .xlsx, Google Sheets), so we can easily import any data that we download into the same folder as our code.

In [ ]:
census = pd.read_csv('census_sample_CA.csv')#, encoding= 'cp1252') #encoding parameter is not usually needed!
census.head()

Let's examine the size of the dataset and see what we are working with in this dataframe.

In [ ]:
census.shape
#We have 1000 rows (1000 individual student entries) and 62 columns

Let's get a list of all 60 columns in the dataset.

In [ ]:
census.columns

We can select a specific column in the dataset using its column name.

In [ ]:
census['Data Year']

Series have a nice function called [value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html) that returns a series listing the number of times each value appears in the series.

In [ ]:
census['Data Year'].value_counts()

Oftentimes, we need to slice the dataframe to obtain smaller subsets of the table to analyze. This can be accomplished by using label (loc) or index (iloc).

### Selection Using Label/Index (using loc)

**Column Selection** 

To select a column of a `DataFrame` by column label, the safest and fastest way is to use the `.loc` [method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.loc.html). General usage of `.loc` looks like `df.loc[rowname, colname]`. (Reminder that the colon `:` means "everything.")  For example, if we want the `color` column of the `ex` data frame, we would use: `ex.loc[:, 'color']`

- You can also slice across columns. For example, `census.loc[:, 'Gender':]` would select the column `Gender` and all columns after `Gender`.

**Row Selection**

Similarly, if we want to select a row by its label, we can use the same `.loc` method. In this case, the "label" of each row refers to the index (ie. primary key) of the dataframe.

In [ ]:
census.loc[1:5, 'Gender']

To get a single column, you can just pass in that column name and it will return a Series. If you need multiple columns or to return a dataframe, you must put the column parameneter in a list.

In [ ]:
census.loc[1:5, ['Gender']]

In [ ]:
census.loc[1:5, ['Gender', 'Height']]

### Selection using Integer location (using iloc)

Pandas has another feature `iloc[]` which lets you slice the dataframe by row position and column position instead of by row index and column label (which is the case for `loc[]`). This is really the main difference between the 2 functions and it is **important** that you remember the difference and why you might want to use one over the other. In addition, with `iloc[]`, the end index is NOT included, like with normal Python slicing.

Note that when you sort or rearrange the dataframe, the *position* of a row is not necessarily equal to the *index* of a row. For example, the first row is not necessarily the row associated with index 1. This distinction is important in understanding the different between `loc[]` and `iloc[]`.

In [ ]:
sorted_census = census.sort_values(by = ['Class Grade'])
sorted_census.head()

Here is an example of how we would get the 2nd, 3rd, and 4th rows with only the `Gender` column of the `census` dataframe using both `iloc[]` and `loc[]`. Observe the difference, especially after sorting by `ClassGrade`.

In [ ]:
sorted_census.iloc[1:4, 3]

In [ ]:
sorted_census.loc[[764,36,34], ['Class Grade (cat)']]

In [ ]:
sorted_census.iloc[1:4, :6]

Lastly, we can change the index of a dataframe using the [set_index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html) method.

In [ ]:
change_index = census.set_index('Gender')
change_index.head()

### Filtering with boolean arrays

Filtering is the process of removing unwanted material.  To clean data, you will have to filter your data at some point: whether it be for clearing up cases with missing values, taking out outliers, or for analyzing subgroups of your data set.  Note that compound expressions have to be grouped with parentheses. Example usage looks like `df[df['column name'] < 5]]`.

Some commonly used comparison operators are given below.

Symbol | Usage      | Meaning 
------ | ---------- | -------------------------------------
==   | a == b   | Does a equal b?
<=   | a <= b   | Is a less than or equal to b?
>=   | a >= b   | Is a greater than or equal to b?
<    | a < b    | Is a less than b?
&#62;    | a &#62; b    | Is a greater than b?
~    | ~p       | Returns negation of p
&#124; | p &#124; q | p OR q
&    | p & q    | p AND q
^  | p ^ q | p XOR q (exclusive or)

In [ ]:
#Select the rows of the Dataframe where Data Year = 2020
census2020 = census[census['Data Year'] == 2020]
census2020.head()

You can also put multiple conditions together. Any time you use `p & q` to filter the dataframe, make sure to use `df[df[(p) & (q)]]` or `df.loc[df[(p) & (q)]])`. That is, make sure to wrap conditions with parentheses.

**Remember** that both slicing and `loc` will achieve the same result, it is just that `loc` is typically faster in production. You are free to use whichever one you would like.

In [ ]:
#Select the rows where DataYear = 2020 and Female
census_2020_f = census[(census['Data Year'] == 2020) & (census['Gender'] == 'Female')]
census_2020_f.head()

## Pandas: Aggregation

In [ ]:
census_q = census.loc[:, ['Age', 'Class Grade', 'Languages spoken']]
census_q.head()

### Dataframe Aggregation
When you first come across a large dataset, the best way to get a sense of it is to get summary statistics. The [aggregate](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.aggregate.html) allows you to do this. Using `df.aggregate()` directly on the whole dataframe will apply the aggregation functions on the entire dataframe at once.

Some common built-in Pandas aggregations are provided below:

Aggregation | Description 
------ | ---------- 
`count()`   | Total number of items
`first()`   | First and last item
`mean(), median()`   | Mean and median
`min(), max()`   | Minimum and maximum
`std(), var()`   | Standard deviation and variance
`mad()`   | Mean absolute deviation
`prod()`   | Product of all items
`sum()`   | Sum of all items


In [ ]:
census_q.aggregate(['min', 'mean'])

In [ ]:
#You can also specify different aggregations for each column
census_q.aggregate({'Age': ['std', 'mean'], 'Class Grade': ['median', 'var'], 'Languages spoken': ['max']})

Simple aggregations can give you a flavor of your dataset, but often we would prefer to aggregate conditionally on some label or index: this is implemented in the so-called [groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) operation. When you first apply `groupby()` to a dataframe, it returns a groupby object.

In [ ]:
census.groupby('Class Grade')

From there, you can specify how you want to aggregate the rows of the Groupby object.

In [ ]:
census.groupby('Class Grade').agg(sum)

As you can see, the agggregation is then applied to each column that the aggregation function can be used on. `sum` can only be applied on integers and floats, so it omits and columns that have other datatypes.

In [ ]:
census.groupby('Class Grade').agg(list).head()

## Visualizations

After exploring your dataset, it's important to be able to represent it visually. `matplotlib.pyplot` is a library that allows you to make basic visualizations. Later in the course, we will explore more advanced libraries to create more complex visualizations.

In [ ]:
import matplotlib as plt

In [ ]:
#Age and foot length for all males in the census dataframe
census_af = census[census['Gender'] == 'Male'].loc[:,['Age', 'Right foot length']]
census_af['Right foot length'] = pd.to_numeric(census_af['Right foot length'])
census_af

You can create a plot directly from the dataframe with the [plt.plot](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.plot.html) by specifying the columns that are to be used for the x and y axes.

In [ ]:
census_af.plot(x = 'Age', y = 'Right foot length', kind = 'scatter')